In [1]:
import lime
import lime.lime_tabular
import collections
import sklearn
import numpy as np
import os
import copy
import string
from io import open
import json

In [2]:
from __future__ import print_function
import io


""" ~~~~~ Used in class AnchorBaseBeam(object): ~~~~~ """
class AnchorExplanation:
    """Object returned by explainers"""
    def __init__(self, type_, exp_map, as_html):
        self.type = type_
        self.exp_map = exp_map
        self.as_html_fn = as_html
    
    """ Not in the main code, just when printting the rages of the anchor's features"""
    def names(self, partial_index=None):
        """
        Returns a list of the names of the anchor conditions.
        Args:
            partial_index (int): lets you get the anchor until a certain index.
            For example, if the anchor is (A=1,B=2,C=2) and partial_index=1,
            this will return ["A=1", "B=2"]
        """
        names = self.exp_map['names']
        if partial_index is not None:
            names = names[:partial_index + 1]
        return names
    
    """ Not in the main code, returns the index of the anchor's features used"""
    def features(self, partial_index=None):
        """
        Returns a list of the features used in the anchor conditions.
        Args:
            partial_index (int): lets you get the anchor until a certain index.
            For example, if the anchor uses features (1, 2, 3) and
            partial_index=1, this will return [1, 2]
        """
        features = self.exp_map['feature']
        if partial_index is not None:
            features = features[:partial_index + 1]
        return features
    
    """ Not in the main code, returns precision of the anchor's"""
    def precision(self, partial_index=None):
        """
        Returns the anchor precision (a float)
        Args:
            partial_index (int): lets you get the anchor precision until a
            certain index. For example, if the anchor has precisions
            [0.1, 0.5, 0.95] and partial_index=1, this will return 0.5
        """
        precision = self.exp_map['precision']
        if len(precision) == 0:
            return self.exp_map['all_precision']
        if partial_index is not None:
            return precision[partial_index]
        else:
            return precision[-1]

    """ Not in the main code, returns coverage of the anchor's """
    def coverage(self, partial_index=None):
        """
        Returns the anchor coverage (a float)
        Args:
            partial_index (int): lets you get the anchor coverage until a
            certain index. For example, if the anchor has coverages
            [0.1, 0.5, 0.95] and partial_index=1, this will return 0.5
        """
        coverage = self.exp_map['coverage']
        if len(coverage) == 0:
            return 1
        if partial_index is not None:
            return coverage[partial_index]
        else:
            return coverage[-1]

In [3]:
"""  anchor_base.py 
     https://github.com/marcotcr/anchor/blob/master/anchor/anchor_base.py  """ 
from __future__ import print_function
# import numpy as np
import operator
import copy
# import sklearn
# import collections

""" Used in def get_anchor_from_tuple"""
def matrix_subset(matrix, n_samples):
    if matrix.shape[0] == 0:
        return matrix
    n_samples = min(matrix.shape[0], n_samples)
    return matrix[np.random.choice(matrix.shape[0], n_samples, replace=False)]

""" ~~~~ Used in calss AnchorTabularExplainer ~~~~ """
class AnchorBaseBeam(object):
    def __init__(self):
        pass
    
    """ Used in dlow_bernoulli """
    @staticmethod
    def kl_bernoulli(p, q):
        p = min(0.9999999999999999, max(0.0000001, p))
        q = min(0.9999999999999999, max(0.0000001, q))
        return (p * np.log(float(p) / q) + (1 - p) *
                np.log(float(1 - p) / (1 - q)))
    
    """ Used in anchor_beam """
    @staticmethod
    def compute_beta(n_features, t, delta):
        alpha = 1.1
        k = 405.5
        temp = np.log(k * n_features * (t ** alpha) / delta)
        return temp + np.log(temp)
    
    """ Used in lucb and anchor_beam """
    @staticmethod
    def dup_bernoulli(p, level):
        lm = p
        um = min(min(1, p + np.sqrt(level / 2.)), 1)
        qm = (um + lm) / 2.
#         print 'lm', lm, 'qm', qm, kl_bernoulli(p, qm)
        if AnchorBaseBeam.kl_bernoulli(p, qm) > level:
            um = qm
        else:
            lm = qm
        return um
    """ Used in anchor_beam """
    @staticmethod
    def lucb(sample_fns, initial_stats, epsilon, delta, batch_size, top_n,
             verbose=False, verbose_every=1):
        # initial_stats must have n_samples, positive
        n_features = len(sample_fns)
        n_samples = np.array(initial_stats['n_samples'])
        positives = np.array(initial_stats['positives'])
        ub = np.zeros(n_samples.shape)
        lb = np.zeros(n_samples.shape)
        for f in np.where(n_samples == 0)[0]:
            n_samples[f] += 1
            positives[f] += sample_fns[f](1)
        if n_features == top_n:
            return range(n_features)
        means = positives / n_samples
        t = 1

        def update_bounds(t):
            sorted_means = np.argsort(means)
            beta = AnchorBaseBeam.compute_beta(n_features, t, delta)
            J = sorted_means[-top_n:]
            not_J = sorted_means[:-top_n]
            for f in not_J:
                ub[f] = AnchorBaseBeam.dup_bernoulli(means[f], beta /
                                                     n_samples[f])
            for f in J:
                lb[f] = AnchorBaseBeam.dlow_bernoulli(means[f],
                                                      beta / n_samples[f])
            ut = not_J[np.argmax(ub[not_J])]
            lt = J[np.argmin(lb[J])]
            return ut, lt
        ut, lt = update_bounds(t)
        B = ub[ut] - lb[lt]
        verbose_count = 0
        while B > epsilon:
            verbose_count += 1
            if verbose and verbose_count % verbose_every == 0:
                print('Best: %d (mean:%.10f, n: %d, lb:%.4f)' %
                      (lt, means[lt], n_samples[lt], lb[lt]), end=' ')
                print('Worst: %d (mean:%.4f, n: %d, ub:%.4f)' %
                      (ut, means[ut], n_samples[ut], ub[ut]), end=' ')
                print('B = %.2f' % B)
            n_samples[ut] += batch_size
            positives[ut] += sample_fns[ut](batch_size)
            means[ut] = positives[ut] / n_samples[ut]
            n_samples[lt] += batch_size
            positives[lt] += sample_fns[lt](batch_size)
            means[lt] = positives[lt] / n_samples[lt]
            t += 1
            ut, lt = update_bounds(t)
            B = ub[ut] - lb[lt]
        sorted_means = np.argsort(means)
        return sorted_means[-top_n:]
    
    """ Used in anchor_beam """
    @staticmethod
    def dlow_bernoulli(p, level):
        um = p
        lm = max(min(1, p - np.sqrt(level / 2.)), 0)
        qm = (um + lm) / 2.
#         print 'lm', lm, 'qm', qm, kl_bernoulli(p, qm)
        if AnchorBaseBeam.kl_bernoulli(p, qm) > level:
            lm = qm
        else:
            um = qm
        return lm

    """ Used in anchor_beam """
    @staticmethod
    def make_tuples(previous_best, state):
        # alters state, computes support for new tuples
        normalize_tuple = lambda x: tuple(sorted(set(x)))  # noqa
        all_features = range(state['n_features'])
        coverage_data = state['coverage_data']
        current_idx = state['current_idx']
        data = state['data'][:current_idx]
        labels = state['labels'][:current_idx]
        if len(previous_best) == 0:
            tuples = [(x, ) for x in all_features]
            for x in tuples:
                pres = data[:, x[0]].nonzero()[0]
                # NEW
                state['t_idx'][x] = set(pres)
                state['t_nsamples'][x] = float(len(pres))
                state['t_positives'][x] = float(labels[pres].sum())
                state['t_order'][x].append(x[0])
                # NEW
                state['t_coverage_idx'][x] = set(
                    coverage_data[:, x[0]].nonzero()[0])
                state['t_coverage'][x] = (
                    float(len(state['t_coverage_idx'][x])) /
                    coverage_data.shape[0])
            return tuples
        new_tuples = set()
        for f in all_features:
            for t in previous_best:
                new_t = normalize_tuple(t + (f, ))
                if len(new_t) != len(t) + 1:
                    continue
                if new_t not in new_tuples:
                    new_tuples.add(new_t)
                    state['t_order'][new_t] = copy.deepcopy(state['t_order'][t])
                    state['t_order'][new_t].append(f)
                    state['t_coverage_idx'][new_t] = (
                        state['t_coverage_idx'][t].intersection(
                            state['t_coverage_idx'][(f,)]))
                    state['t_coverage'][new_t] = (
                        float(len(state['t_coverage_idx'][new_t])) /
                        coverage_data.shape[0])
                    t_idx = np.array(list(state['t_idx'][t]))
                    t_data = state['data'][t_idx]
                    present = np.where(t_data[:, f] == 1)[0]
                    state['t_idx'][new_t] = set(t_idx[present])
                    idx_list = list(state['t_idx'][new_t])
                    state['t_nsamples'][new_t] = float(len(idx_list))
                    state['t_positives'][new_t] = np.sum(
                        state['labels'][idx_list])
        return list(new_tuples)
    
    """ Used in anchor_beam """
    @staticmethod
    def get_sample_fns(sample_fn, tuples, state):
        # each sample fn returns number of positives
        sample_fns = []
        def complete_sample_fn(t, n):
            raw_data, data, labels = sample_fn(list(t), n)
            current_idx = state['current_idx']
            # idxs = range(state['data'].shape[0], state['data'].shape[0] + n)
            idxs = range(current_idx, current_idx + n)
            state['t_idx'][t].update(idxs)
            state['t_nsamples'][t] += n
            state['t_positives'][t] += labels.sum()
            state['data'][idxs] = data
            state['raw_data'][idxs] = raw_data
            state['labels'][idxs] = labels
            state['current_idx'] += n
            if state['current_idx'] >= state['data'].shape[0] - max(1000, n):
                prealloc_size = state['prealloc_size']
                current_idx = data.shape[0]
                state['data'] = np.vstack(
                    (state['data'],
                     np.zeros((prealloc_size, data.shape[1]), data.dtype)))
                state['raw_data'] = np.vstack(
                    (state['raw_data'],
                     np.zeros((prealloc_size, raw_data.shape[1]),
                              raw_data.dtype)))
                state['labels'] = np.hstack(
                    (state['labels'],
                     np.zeros(prealloc_size, labels.dtype)))
            # This can be really slow
            # state['data'] = np.vstack((state['data'], data))
            # state['raw_data'] = np.vstack((state['raw_data'], raw_data))
            # state['labels'] = np.hstack((state['labels'], labels))
            return labels.sum()
        for t in tuples:
            sample_fns.append(lambda n, t=t: complete_sample_fn(t, n))
        return sample_fns
    
    """ Used in anchor_beam """
    @staticmethod
    def get_initial_statistics(tuples, state):
        stats = {
            'n_samples': [],
            'positives': []
        }
        for t in tuples:
            stats['n_samples'].append(state['t_nsamples'][t])
            stats['positives'].append(state['t_positives'][t])
        return stats
    
    """ Used in anchor_beam """
    @staticmethod
    def get_anchor_from_tuple(t, state):
        # TODO: This is wrong, some of the intermediate anchors may not exist.
        anchor = {'feature': [], 'mean': [], 'precision': [],
                  'coverage': [], 'examples': [], 'all_precision': 0}
        anchor['num_preds'] = state['data'].shape[0]
        normalize_tuple = lambda x: tuple(sorted(set(x)))  # noqa
        current_t = tuple()
        for f in state['t_order'][t]:
            current_t = normalize_tuple(current_t + (f,))

            mean = (state['t_positives'][current_t] /
                    state['t_nsamples'][current_t])
            anchor['feature'].append(f)
            anchor['mean'].append(mean)
            anchor['precision'].append(mean)
            anchor['coverage'].append(state['t_coverage'][current_t])
            raw_idx = list(state['t_idx'][current_t])
            raw_data = state['raw_data'][raw_idx]
            covered_true = (
                state['raw_data'][raw_idx][state['labels'][raw_idx] == 1])
            covered_false = (
                state['raw_data'][raw_idx][state['labels'][raw_idx] == 0])
            exs = {}
            exs['covered'] = matrix_subset(raw_data, 10)
            exs['covered_true'] = matrix_subset(covered_true, 10)
            exs['covered_false'] = matrix_subset(covered_false, 10)
            exs['uncovered_true'] = np.array([])
            exs['uncovered_false'] = np.array([])
            anchor['examples'].append(exs)
        return anchor
    
    """ I think is where the best anchor is selected (using mult)"""
    @staticmethod
    def anchor_beam(sample_fn, delta=0.05, epsilon=0.1, batch_size=10,
                    min_shared_samples=0, desired_confidence=1, beam_size=1,
                    verbose=False, epsilon_stop=0.05, min_samples_start=0,
                    max_anchor_size=None, verbose_every=1,
                    stop_on_first=False, coverage_samples=10000):
        anchor = {'feature': [], 'mean': [], 'precision': [],
                  'coverage': [], 'examples': [], 'all_precision': 0}
        _, coverage_data, _ = sample_fn([], coverage_samples, compute_labels=False)
        raw_data, data, labels = sample_fn([], max(1, min_samples_start))
        mean = labels.mean()
        beta = np.log(1. / delta)
        lb = AnchorBaseBeam.dlow_bernoulli(mean, beta / data.shape[0])
        while mean > desired_confidence and lb < desired_confidence - epsilon:
            nraw_data, ndata, nlabels = sample_fn([], batch_size)
            data = np.vstack((data, ndata))
            raw_data = np.vstack((raw_data, nraw_data))
            labels = np.hstack((labels, nlabels))
            mean = labels.mean()
            lb = AnchorBaseBeam.dlow_bernoulli(mean, beta / data.shape[0])
        if lb > desired_confidence:
            anchor['num_preds'] = data.shape[0]
            anchor['all_precision'] = mean
            return anchor
        prealloc_size = batch_size * 10000
        current_idx = data.shape[0]
        data = np.vstack((data, np.zeros((prealloc_size, data.shape[1]),
                                         data.dtype)))
        raw_data = np.vstack(
            (raw_data, np.zeros((prealloc_size, raw_data.shape[1]),
                                raw_data.dtype)))
        labels = np.hstack((labels, np.zeros(prealloc_size, labels.dtype)))
        n_features = data.shape[1]
        state = {'t_idx': collections.defaultdict(lambda: set()),
                 't_nsamples': collections.defaultdict(lambda: 0.),
                 't_positives': collections.defaultdict(lambda: 0.),
                 'data': data,
                 'prealloc_size': prealloc_size,
                 'raw_data': raw_data,
                 'labels': labels,
                 'current_idx': current_idx,
                 'n_features': n_features,
                 't_coverage_idx': collections.defaultdict(lambda: set()),
                 't_coverage': collections.defaultdict(lambda: 0.),
                 'coverage_data': coverage_data,
                 't_order': collections.defaultdict(lambda: list())
                 }
        current_size = 1
        best_of_size = {0: []}
        best_coverage = -1
        best_tuple = ()
        t = 1
        if max_anchor_size is None:
            max_anchor_size = n_features
        while current_size <= max_anchor_size:
            tuples = AnchorBaseBeam.make_tuples(
                best_of_size[current_size - 1], state)
            tuples = [x for x in tuples
                      if state['t_coverage'][x] > best_coverage]
            if len(tuples) == 0:
                break
            sample_fns = AnchorBaseBeam.get_sample_fns(sample_fn, tuples,
                                                       state)
            initial_stats = AnchorBaseBeam.get_initial_statistics(tuples,
                                                                  state)
            # print tuples, beam_size
            chosen_tuples = AnchorBaseBeam.lucb(
                sample_fns, initial_stats, epsilon, delta, batch_size,
                min(beam_size, len(tuples)),
                verbose=verbose, verbose_every=verbose_every)
            best_of_size[current_size] = [tuples[x] for x in chosen_tuples]
            if verbose:
                print('Best of size ', current_size, ':')
            # print state['data'].shape[0]
            stop_this = False
            for i, t in zip(chosen_tuples, best_of_size[current_size]):
                # I can choose at most (beam_size - 1) tuples at each step,
                # and there are at most n_feature steps
                beta = np.log(1. /
                              (delta / (1 + (beam_size - 1) * n_features)))
                # beta = np.log(1. / delta)
                # if state['t_nsamples'][t] == 0:
                #     mean = 1
                # else:
                mean = state['t_positives'][t] / state['t_nsamples'][t]
                lb = AnchorBaseBeam.dlow_bernoulli(
                    mean, beta / state['t_nsamples'][t])
                ub = AnchorBaseBeam.dup_bernoulli(
                    mean, beta / state['t_nsamples'][t])
                coverage = state['t_coverage'][t]
                if verbose:
                    print(i, mean, lb, ub)
                while ((mean >= desired_confidence and
                       lb < desired_confidence - epsilon_stop) or
                       (mean < desired_confidence and
                        ub >= desired_confidence + epsilon_stop)):
                    # print mean, lb, state['t_nsamples'][t]
                    sample_fns[i](batch_size)
                    mean = state['t_positives'][t] / state['t_nsamples'][t]
                    lb = AnchorBaseBeam.dlow_bernoulli(
                        mean, beta / state['t_nsamples'][t])
                    ub = AnchorBaseBeam.dup_bernoulli(
                        mean, beta / state['t_nsamples'][t])
                if verbose:
                    print('%s mean = %.2f lb = %.2f ub = %.2f coverage: %.2f n: %d' % (t, mean, lb, ub, coverage, state['t_nsamples'][t]))
                if mean >= desired_confidence and lb > desired_confidence - epsilon_stop:
                    if verbose:
                        print('Found eligible anchor ', t, 'Coverage:',
                              coverage, 'Is best?', coverage > best_coverage)
                    if coverage > best_coverage:
                        best_coverage = coverage
                        best_tuple = t
                        if best_coverage == 1 or stop_on_first:
                            stop_this = True
            if stop_this:
                break
            current_size += 1
        if best_tuple == ():
            # Could not find an anchor, will now choose the highest precision
            # amongst the top K from every round
            if verbose:
                print('Could not find an anchor, now doing best of each size')
            tuples = []
            for i in range(0, current_size):
                tuples.extend(best_of_size[i])
            # tuples = best_of_size[current_size - 1]
            sample_fns = AnchorBaseBeam.get_sample_fns(sample_fn, tuples,
                                                       state)
            initial_stats = AnchorBaseBeam.get_initial_statistics(tuples,
                                                                  state)
            # print tuples, beam_size
            chosen_tuples = AnchorBaseBeam.lucb(
                sample_fns, initial_stats, epsilon, delta, batch_size,
                1, verbose=verbose)
            best_tuple = tuples[chosen_tuples[0]]
        # return best_tuple, state
        return AnchorBaseBeam.get_anchor_from_tuple(best_tuple, state)

In [307]:
""" anchor_tabular.py 
    https://github.com/marcotcr/anchor/blob/master/anchor/anchor_tabular.py"""

"""def as_html(, def jsonize(x)"""

def id_generator(size=15):
    """Helper function to generate random div ids. This is useful for embedding
    HTML into ipython notebooks."""
    chars = list(string.ascii_uppercase + string.digits)

    
    
""" This is the main class-file to compute the anchors for tabular data """
class AnchorTabularExplainer(object):
    """
        Args:
            class_names: list of strings
            feature_names: list of strings
            train_data: used to sample (bootstrap)
            categorical_names: map from integer to list of strings, names for each
                value of the categorical features. Every feature that is not in
                this map will be considered as ordinal or continuous, and thus discretized.
        """

    def __init__(self, class_names, feature_names, train_data,
                 categorical_names={}, discretizer='quartile', encoder_fn=None):
        self.min = {}
        self.max = {}
        self.disc = collections.namedtuple('random_name2',
                                              ['discretize'])(lambda x: x)
        self.encoder_fn = lambda x: x
        if encoder_fn is not None:
            self.encoder_fn = encoder_fn
        self.categorical_features = []
        self.feature_names = feature_names
        self.train = train_data
        self.class_names = class_names
        self.categorical_names = copy.deepcopy(categorical_names)
        if categorical_names:
            self.categorical_features = sorted(categorical_names.keys())

        if discretizer == 'quartile':
            self.disc = lime.lime_tabular.QuartileDiscretizer(train_data,
                                                         self.categorical_features,
                                                         self.feature_names)
        elif discretizer == 'decile':
            self.disc = lime.lime_tabular.DecileDiscretizer(train_data,
                                                     self.categorical_features,
                                                     self.feature_names)
        else:
            raise ValueError('Discretizer must be quartile or decile')

        self.ordinal_features = [x for x in range(len(feature_names)) if x not in self.categorical_features]

        self.d_train = self.disc.discretize(self.train)
        self.categorical_names.update(self.disc.names)
        self.categorical_features += self.ordinal_features

        for f in range(train_data.shape[1]):
            self.min[f] = np.min(train_data[:, f])
            self.max[f] = np.max(train_data[:, f])
            
    
    """ This is def explain_instance: def sample_fn """
    def sample_from_train(self, conditions_eq, conditions_neq, conditions_geq,
                          conditions_leq, num_samples):
        """
        bla
        """
        train = self.train
        d_train = self.d_train
        idx = np.random.choice(range(train.shape[0]), num_samples,
                               replace=True)
        sample = train[idx]
        d_sample = d_train[idx]
        for f in conditions_eq:
            sample[:, f] = np.repeat(conditions_eq[f], num_samples)
        for f in conditions_geq:
            idx = d_sample[:, f] <= conditions_geq[f]
            if f in conditions_leq:
                idx = (idx + (d_sample[:, f] > conditions_leq[f])).astype(bool)
            if idx.sum() == 0:
                continue
            options = d_train[:, f] > conditions_geq[f]
            if f in conditions_leq:
                options = options * (d_train[:, f] <= conditions_leq[f])
            if options.sum() == 0:
                min_ = conditions_geq.get(f, self.min[f])
                max_ = conditions_leq.get(f, self.max[f])
                to_rep = np.random.uniform(min_, max_, idx.sum())
            else:
                to_rep = np.random.choice(train[options, f], idx.sum(),
                                          replace=True)
            sample[idx, f] = to_rep
        for f in conditions_leq:
            if f in conditions_geq:
                continue
            idx = d_sample[:, f] > conditions_leq[f]
            if idx.sum() == 0:
                continue
            options = d_train[:, f] <= conditions_leq[f]
            if options.sum() == 0:
                min_ = conditions_geq.get(f, self.min[f])
                max_ = conditions_leq.get(f, self.max[f])
                to_rep = np.random.uniform(min_, max_, idx.sum())
            else:
                to_rep = np.random.choice(train[options, f], idx.sum(),
                                          replace=True)
            sample[idx, f] = to_rep
        return sample

    """ Used in get_sample_fn: def sample_fn """
    def as_html(self, exp, **kwargs):
        """bla"""
        exp_map = self.to_explanation_map(exp)

#         def jsonize(x): return json.dumps(x)
#         this_dir, _ = os.path.split(__file__)
#         bundle = open(os.path.join(this_dir, 'bundle.js'), encoding='utf8').read()
#         random_id = 'top_div' + id_generator()
#         out = u'''<html>
#         <meta http-equiv="content-type" content="text/html; charset=UTF8">
#         <head><script>%s </script></head><body>''' % bundle
#         out += u'''
#         <div id="{random_id}" />
#         <script>
#             div = d3.select("#{random_id}");
#             lime.RenderExplanationFrame(div,{label_names}, {predict_proba},
#             {true_class}, {explanation}, {raw_data}, "tabular", {explanation_type});
#         </script>'''.format(random_id=random_id,
#                             label_names=jsonize(exp_map['labelNames']),
#                             predict_proba=jsonize(exp_map['predictProba']),
#                             true_class=jsonize(exp_map['trueClass']),
#                             explanation=jsonize(exp_map['explanation']),
#                             raw_data=jsonize(exp_map['rawData']),
#                             explanation_type=jsonize(exp_map['explanationType']))
#         out += u'</body></html>'
        return out
    
    """ Used in def explain_instance """
    def get_sample_fn(self, data_row, classifier_fn, desired_label=None):
        
        """predict_fn(x) its the prediction of the blackbox model"""
        def predict_fn(x):
            return classifier_fn(self.encoder_fn(x))
        true_label = desired_label
#         print("true_label", true_label)
        if true_label is None:
            true_label = predict_fn(data_row.reshape(1, -1))[0]
#             print("data_row", data_row)
#             print("data_row.reshape(1, -1)", data_row.reshape(1, -1)[0])
#             print("true_label in loop", true_label)
        # must map present here to include categorical features (for conditions_eq), and numerical features for geq and leq
        """ makes a map dictionary saving if checking from 0 to 3 if the location of the quartile is <= or > otherwise,
            for each of the features"""
        mapping = {}
        """ Discretize the features into quartiles """
        data_row = self.disc.discretize(data_row.reshape(1, -1))[0]
        
        """self.categorical_feature = self.ordinal_features: are list of the number of (n) features
            from 0 to n-1: [0, 1, 2, 3],  v goes from 0 to 3"""
        for f in self.categorical_features:
            if f in self.ordinal_features:
                for v in range(len(self.categorical_names[f])):
                    idx = len(mapping)
                    print("mapping:", mapping)
                    print("idx = len(mapping):",  idx)
                    print("\n f, v:", f, v)
                    print("data_row:",  data_row)
                    print("data_row[f]:",  data_row[f])
                    print("data_row[f] <= v", data_row[f] <= v)
                    print("v != len(self.categorical_names[f]) - 1:",  v != len(self.categorical_names[f]) - 1)
                    if data_row[f] <= v and v != len(self.categorical_names[f]) - 1:
                        mapping[idx] = (f, 'leq', v)
                        # names[idx] = '%s <= %s' % (self.feature_names[f], v)
                    elif data_row[f] > v:
                        mapping[idx] = (f, 'geq', v)
                        # names[idx] = '%s > %s' % (self.feature_names[f], v)
            else:
                print("RICHARD THINKS I WILL NEVER BE PRINTED SINCE F WILL ALWAYS BE IN self.ordinal_features")
                idx = len(mapping)
                mapping[idx] = (f, 'eq', data_row[f])
            # names[idx] = '%s = %s' % (
            #     self.feature_names[f],
            #     self.categorical_names[f][int(data_row[f])])

        def sample_fn(present, num_samples, compute_labels=True):
            conditions_eq = {}
            conditions_leq = {}
            conditions_geq = {}
            
            print("present:", present)
            for x in present:
                f, op, v = mapping[x]
                if op == 'eq':
                    conditions_eq[f] = v
                if op == 'leq':
                    if f not in conditions_leq:
                        conditions_leq[f] = v
                    conditions_leq[f] = min(conditions_leq[f], v)
                if op == 'geq':
                    if f not in conditions_geq:
                        conditions_geq[f] = v
                    conditions_geq[f] = max(conditions_geq[f], v)
            # conditions_eq = dict([(x, data_row[x]) for x in present])
            print("conditions_eq", conditions_eq)
            print("conditions_leq", conditions_leq)
            print("conditions_geq", conditions_geq)
            raw_data = self.sample_from_train(
                conditions_eq, {}, conditions_geq, conditions_leq, num_samples)
            d_raw_data = self.disc.discretize(raw_data)
            print("raw_data", raw_data)
            print("d_raw_data", d_raw_data)
            data = np.zeros((num_samples, len(mapping)), int)
            for i in mapping:
                f, op, v = mapping[i]
                if op == 'eq':
                    data[:, i] = (d_raw_data[:, f] == data_row[f]).astype(int)
                if op == 'leq':
                    data[:, i] = (d_raw_data[:, f] <= v).astype(int)
                if op == 'geq':
                    data[:, i] = (d_raw_data[:, f] > v).astype(int)
            # data = (raw_data == data_row).astype(int)
            labels = []
            if compute_labels:
                labels = (predict_fn(raw_data) == true_label).astype(int)
            return raw_data, data, labels
        return sample_fn, mapping
         
    """ Used outside to explain an instance, this is the core functin, where all of the explanabily can happends ****"""
    def explain_instance(self, data_row, classifier_fn, threshold=0.95,
                          delta=0.1, tau=0.15, batch_size=100,
                          max_anchor_size=None,
                          desired_label=None,
                          beam_size=4, **kwargs):
        # It's possible to pass in max_anchor_size
        sample_fn, mapping = self.get_sample_fn(
            data_row, classifier_fn, desired_label=desired_label)
        # return sample_fn, mapping
        """ Changing exp = anchor_base.AnchorBaseBeam.anchor_beam(...) 
                to   exp = AnchorBaseBeam.anchor_beam(...) 
                since class is not imported any more """
        exp = AnchorBaseBeam.anchor_beam(
            sample_fn, delta=delta, epsilon=tau, batch_size=batch_size,
            desired_confidence=threshold, max_anchor_size=max_anchor_size,
            **kwargs)
        self.add_names_to_exp(data_row, exp, mapping)
        exp['instance'] = data_row
        exp['prediction'] = classifier_fn(self.encoder_fn(data_row.reshape(1, -1)))[0]
        
        """ cahnge: explanation = anchor_explanation.AnchorExplanation('tabular', exp, self.as_html)
                to: explanation = AnchorExplanation('tabular', exp, self.as_html) 
                Not imported but class defined """
        explanation = AnchorExplanation('tabular', exp, self.as_html)
        return explanation
    
    """ Used in explain_instance """
    def add_names_to_exp(self, data_row, hoeffding_exp, mapping):
        # TODO: precision recall is all wrong, coverage functions wont work
        # anymore due to ranges
        idxs = hoeffding_exp['feature']
        hoeffding_exp['names'] = []
        hoeffding_exp['feature'] = [mapping[idx][0] for idx in idxs]
        ordinal_ranges = {}
        for idx in idxs:
            f, op, v = mapping[idx]
            if op == 'geq' or op == 'leq':
                if f not in ordinal_ranges:
                    ordinal_ranges[f] = [float('-inf'), float('inf')]
            if op == 'geq':
                ordinal_ranges[f][0] = max(ordinal_ranges[f][0], v)
            if op == 'leq':
                ordinal_ranges[f][1] = min(ordinal_ranges[f][1], v)
        handled = set()
        for idx in idxs:
            f, op, v = mapping[idx]
            # v = data_row[f]
            if op == 'eq':
                fname = '%s = ' % self.feature_names[f]
                if f in self.categorical_names:
                    v = int(v)
                    if ('<' in self.categorical_names[f][v]
                            or '>' in self.categorical_names[f][v]):
                        fname = ''
                    fname = '%s%s' % (fname, self.categorical_names[f][v])
                else:
                    fname = '%s%.2f' % (fname, v)
            else:
                if f in handled:
                    continue
                geq, leq = ordinal_ranges[f]
                fname = ''
                geq_val = ''
                leq_val = ''
                if geq > float('-inf'):
                    if geq == len(self.categorical_names[f]) - 1:
                        geq = geq - 1
                    name = self.categorical_names[f][geq + 1]
                    if '<' in name:
                        geq_val = name.split()[0]
                    elif '>' in name:
                        geq_val = name.split()[-1]
                if leq < float('inf'):
                    name = self.categorical_names[f][leq]
                    if leq == 0:
                        leq_val = name.split()[-1]
                    elif '<' in name:
                        leq_val = name.split()[-1]
                if leq_val and geq_val:
                    fname = '%s < %s <= %s' % (geq_val, self.feature_names[f],
                                               leq_val)
                elif leq_val:
                    fname = '%s <= %s' % (self.feature_names[f], leq_val)
                elif geq_val:
                    fname = '%s > %s' % (self.feature_names[f], geq_val)
                handled.add(f)
            hoeffding_exp['names'].append(fname)
   
#     """~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  NOT NEEDED  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""
#     """ This functions in not used at all in the code """
#     def transform_to_examples(self, examples, features_in_anchor=[],
#                               predicted_label=None):
#         ret_obj = []
#         if len(examples) == 0:
#             return ret_obj
#         weights = [int(predicted_label) if x in features_in_anchor else -1
#                    for x in range(examples.shape[1])]
#         examples = self.disc.discretize(examples)
#         for ex in examples:
#             values = [self.categorical_names[i][int(ex[i])]
#                       if i in self.categorical_features
#                       else ex[i] for i in range(ex.shape[0])]
#             ret_obj.append(list(zip(self.feature_names, values, weights)))
#         return ret_obj
    

#     """~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ END - NOT NEEDED  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""

# # """ We do not use  def transform_to_examples and  def to_explanation_map(self, exp)"""

In [308]:
# Checking if these two function is sufficient to implement a simple ANCHOR in the iris data set.

In [309]:
import fatf
import fatf.utils.data.datasets as fatf_datasets

iris_data_dict = fatf_datasets.load_iris()
iris_data = iris_data_dict['data']
iris_target = iris_data_dict['target']
iris_feature_names = iris_data_dict['feature_names'].tolist()
iris_target_names = iris_data_dict['target_names'].tolist()

In [310]:
import sklearn.ensemble

train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris_data, iris_target, train_size=0.80)
blackbox_model = sklearn.ensemble.RandomForestClassifier(n_estimators=20)
blackbox_model.fit(train, labels_train)
print('Train', sklearn.metrics.accuracy_score(labels_train, blackbox_model.predict(train)))
print('Test', sklearn.metrics.accuracy_score(labels_test, blackbox_model.predict(test)))

Train 1.0
Test 0.9666666666666667


In [311]:



explainer = AnchorTabularExplainer(
    iris_target_names,
    iris_feature_names,
    train)

In [312]:
test[idx]


array([5.5, 2.4, 3.7, 1. ], dtype=float32)

In [313]:
idx = 17
np.random.seed(1)
print('Prediction: ', explainer.class_names[int(blackbox_model.predict(test[idx].reshape(1, -1))[0])])
""" cahnge exp = explainer.explain_instance(...) 
    to AnchorExplanation """
exp = explainer.explain_instance(test[idx], blackbox_model.predict, threshold=0.95)

Prediction:  versicolor
mapping: {}
idx = len(mapping): 0

 f, v: 0 0
data_row: [1. 0. 1. 1.]
data_row[f]: 1.0
data_row[f] <= v False
v != len(self.categorical_names[f]) - 1: True
mapping: {0: (0, 'geq', 0)}
idx = len(mapping): 1

 f, v: 0 1
data_row: [1. 0. 1. 1.]
data_row[f]: 1.0
data_row[f] <= v True
v != len(self.categorical_names[f]) - 1: True
mapping: {0: (0, 'geq', 0), 1: (0, 'leq', 1)}
idx = len(mapping): 2

 f, v: 0 2
data_row: [1. 0. 1. 1.]
data_row[f]: 1.0
data_row[f] <= v True
v != len(self.categorical_names[f]) - 1: True
mapping: {0: (0, 'geq', 0), 1: (0, 'leq', 1), 2: (0, 'leq', 2)}
idx = len(mapping): 3

 f, v: 0 3
data_row: [1. 0. 1. 1.]
data_row[f]: 1.0
data_row[f] <= v True
v != len(self.categorical_names[f]) - 1: False
mapping: {0: (0, 'geq', 0), 1: (0, 'leq', 1), 2: (0, 'leq', 2)}
idx = len(mapping): 3

 f, v: 1 0
data_row: [1. 0. 1. 1.]
data_row[f]: 0.0
data_row[f] <= v True
v != len(self.categorical_names[f]) - 1: True
mapping: {0: (0, 'geq', 0), 1: (0, 'leq', 1),

 [5.7 3.  1.5 0.4]]
d_raw_data [[0. 0. 0. 0.]
 [2. 0. 2. 3.]
 [2. 0. 2. 2.]
 [1. 0. 1. 0.]
 [3. 1. 2. 3.]
 [0. 1. 0. 0.]
 [3. 0. 2. 2.]
 [2. 0. 3. 3.]
 [3. 1. 3. 3.]
 [1. 1. 1. 1.]
 [0. 1. 0. 1.]
 [1. 0. 0. 0.]
 [2. 0. 1. 1.]
 [1. 0. 2. 3.]
 [2. 0. 2. 2.]
 [2. 1. 2. 2.]
 [0. 1. 0. 0.]
 [1. 0. 1. 1.]
 [0. 0. 0. 1.]
 [2. 0. 3. 3.]
 [2. 0. 3. 3.]
 [2. 1. 2. 2.]
 [1. 0. 2. 3.]
 [0. 1. 0. 0.]
 [3. 0. 3. 3.]
 [1. 0. 1. 1.]
 [2. 0. 3. 3.]
 [2. 0. 1. 1.]
 [3. 0. 3. 3.]
 [3. 1. 1. 2.]
 [2. 1. 2. 2.]
 [1. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [3. 1. 3. 3.]
 [1. 1. 1. 1.]
 [2. 0. 1. 1.]
 [3. 0. 3. 3.]
 [2. 1. 3. 3.]
 [2. 0. 2. 2.]
 [1. 1. 1. 0.]
 [2. 1. 2. 2.]
 [3. 1. 2. 2.]
 [1. 1. 0. 1.]
 [0. 1. 0. 0.]
 [2. 1. 2. 2.]
 [3. 0. 3. 3.]
 [1. 1. 1. 1.]
 [1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [2. 0. 2. 2.]
 [3. 1. 3. 3.]
 [3. 1. 2. 3.]
 [2. 1. 3. 3.]
 [2. 0. 2. 2.]
 [1. 0. 2. 3.]
 [2. 0. 3. 3.]
 [3. 1. 3. 3.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 1. 1. 1.]
 [3. 1. 3. 2.]
 [2. 1. 2

 [1. 2. 3. 2.]]
present: [9]
conditions_eq {}
conditions_leq {}
conditions_geq {3: 0}
raw_data [[4.6 3.2 1.4 1. ]
 [4.9 3.1 1.5 1.8]
 [6.  3.4 4.5 1.6]
 [6.  2.7 5.1 1.6]
 [4.8 3.4 1.9 2.1]
 [6.1 3.  4.9 1.8]
 [6.5 2.8 4.6 1.5]
 [4.6 3.1 1.5 1.8]
 [5.7 2.5 5.  2. ]
 [5.  3.5 1.6 0.6]
 [5.7 2.5 5.  2. ]
 [5.9 3.2 4.8 1.8]
 [4.8 3.4 1.9 2.1]
 [5.6 3.  4.5 1.5]
 [5.4 3.4 1.7 1.9]
 [6.5 3.  5.2 2. ]
 [7.1 3.  5.9 2.1]
 [5.5 4.2 1.4 1.5]
 [5.6 2.7 4.2 1.3]
 [6.3 3.3 6.  2.5]
 [5.6 3.  4.1 1.3]
 [5.7 2.9 4.2 1.3]
 [5.2 4.1 1.5 1.5]
 [4.7 3.2 1.6 1.6]
 [6.8 3.  5.5 2.1]
 [6.3 2.8 5.1 1.5]
 [5.4 3.4 1.5 0.4]
 [6.5 3.  5.2 2. ]
 [6.7 3.1 4.7 1.5]
 [7.3 2.9 6.3 1.8]
 [4.6 3.1 1.5 0.4]
 [7.3 2.9 6.3 1.8]
 [6.9 3.1 5.1 2.3]
 [6.7 3.  5.  1.7]
 [6.4 2.8 5.6 2.1]
 [6.  3.  4.8 1.8]
 [6.1 3.  4.6 1.4]
 [6.4 3.1 5.5 1.8]
 [6.  2.2 5.  1.5]
 [7.2 3.2 6.  1.8]
 [5.1 3.5 1.4 1. ]
 [6.7 3.  5.2 2.3]
 [5.6 2.7 4.2 1.3]
 [6.3 2.8 5.1 1.5]
 [4.8 3.  1.4 0.4]
 [5.4 3.4 1.5 0.4]
 [4.9 2.5 4.5 1.7]
 [5.4 3.4 1.

present: [11]
conditions_eq {}
conditions_leq {3: 2}
conditions_geq {}
raw_data [[5.6 3.  4.5 1.5]
 [6.9 3.1 4.9 1.5]
 [4.7 3.2 1.6 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [6.4 3.2 4.5 1.5]
 [6.4 2.8 5.6 0.2]
 [6.1 2.8 4.7 1.2]
 [5.4 3.4 1.5 0.4]
 [6.6 3.  4.4 1.4]
 [5.1 3.8 1.5 0.3]
 [5.4 3.9 1.3 0.4]
 [5.  3.  1.6 0.2]
 [6.7 3.  5.2 1.3]
 [5.7 2.9 4.2 1.3]
 [6.2 2.8 4.8 1.8]
 [4.8 3.1 1.6 0.2]
 [6.1 3.  4.6 1.4]
 [4.4 3.2 1.3 0.2]
 [5.1 3.8 1.6 0.2]
 [6.7 3.3 5.7 1. ]
 [5.2 4.1 1.5 0.1]
 [6.7 2.5 5.8 1.8]
 [5.9 3.2 4.8 1.8]
 [4.9 2.5 4.5 1.7]
 [5.4 3.4 1.7 0.2]
 [6.6 2.9 4.6 1.3]
 [5.8 2.7 4.1 1. ]
 [5.8 4.  1.2 0.2]
 [6.8 2.8 4.8 1.4]
 [6.  3.  4.8 1.8]
 [5.6 3.  4.5 1.5]
 [6.  2.2 5.  1.5]
 [5.6 2.9 3.6 1.3]
 [5.6 2.9 3.6 1.3]
 [5.1 3.3 1.7 0.5]
 [7.3 2.9 6.3 1.8]
 [6.3 2.9 5.6 1.8]
 [4.5 2.3 1.3 0.3]
 [6.5 2.8 4.6 1.5]
 [5.8 4.  1.2 0.2]
 [5.1 3.3 1.7 0.5]
 [6.1 3.  4.9 1.8]
 [5.  2.3 3.3 1. ]
 [5.1 3.8 1.5 0.3]
 [6.  2.9 4.5 1.5]
 [5.1 2.5 3.  1.1]
 [5.1 3.8 1.6 0.2]
 [5.2 4.

present: [6]
conditions_eq {}
conditions_leq {}
conditions_geq {2: 0}
raw_data [[5.8 2.6 4.  1.2]
 [6.7 3.1 4.4 1.4]
 [4.6 3.1 5.6 0.2]
 [4.9 3.  3.9 0.2]
 [6.3 2.3 4.4 1.3]
 [7.6 3.  6.6 2.1]
 [6.5 2.8 4.6 1.5]
 [4.8 3.4 1.9 0.2]
 [5.7 2.8 4.5 1.3]
 [6.2 2.8 4.8 1.8]
 [6.5 3.  5.2 2. ]
 [6.3 2.8 5.1 1.5]
 [5.8 2.7 5.1 1.9]
 [4.5 2.3 5.6 0.3]
 [4.9 3.6 1.7 0.1]
 [4.8 3.  4.9 0.3]
 [5.4 3.  4.5 1.5]
 [7.2 3.6 6.1 2.5]
 [4.6 3.2 5.3 0.2]
 [6.1 2.8 4.  1.3]
 [6.9 3.1 5.4 2.1]
 [6.6 3.  4.4 1.4]
 [5.  3.6 5.2 0.2]
 [6.  3.  4.8 1.8]
 [5.2 2.7 3.9 1.4]
 [6.3 2.3 4.4 1.3]
 [6.5 3.  5.2 2. ]
 [6.7 3.3 5.7 2.1]
 [5.  3.5 5.3 0.3]
 [6.3 2.8 5.1 1.5]
 [5.5 2.4 3.8 1.1]
 [5.1 3.5 5.6 0.3]
 [6.6 2.9 4.6 1.3]
 [5.4 3.7 4.5 0.2]
 [5.4 3.4 4.  0.4]
 [5.7 4.4 5.  0.4]
 [4.9 2.5 4.5 1.7]
 [5.  3.6 3.9 0.2]
 [6.4 2.8 5.6 2.1]
 [7.1 3.  5.9 2.1]
 [5.5 2.3 4.  1.3]
 [6.1 2.8 4.7 1.2]
 [6.7 3.3 5.7 2.1]
 [5.8 2.7 5.1 1.9]
 [6.3 2.5 5.  1.9]
 [5.  3.4 4.2 0.2]
 [7.7 3.8 6.7 2.2]
 [6.4 3.1 5.5 1.8]
 [6.2 3.4

raw_data [[4.8 3.4 1.9 0.2]
 [5.5 2.5 4.  1.3]
 [7.7 3.8 1.6 2.2]
 [5.7 3.8 1.7 0.3]
 [7.3 2.9 4.  1.8]
 [6.7 3.1 4.4 1.4]
 [6.  2.2 4.  1. ]
 [5.  3.6 1.4 0.2]
 [6.9 3.1 4.9 2.1]
 [6.7 3.1 4.4 1.4]
 [7.7 3.  1.7 2.3]
 [6.4 2.7 1.3 1.9]
 [5.4 3.9 1.7 0.4]
 [6.2 3.4 4.5 2.3]
 [7.9 3.8 1.6 2. ]
 [4.9 3.1 1.5 0.2]
 [6.5 3.  1.4 1.8]
 [5.4 3.  4.5 1.5]
 [6.7 3.  1.7 2.3]
 [4.5 2.3 1.3 0.3]
 [5.5 2.5 4.  1.3]
 [5.8 2.7 5.1 1.9]
 [6.2 3.4 1.3 2.3]
 [4.5 2.3 1.3 0.3]
 [6.  2.2 4.  1. ]
 [5.4 3.9 1.3 0.4]
 [6.  2.7 5.1 1.6]
 [7.7 2.6 4.  2.3]
 [5.  3.5 1.6 0.6]
 [4.9 3.1 1.5 0.1]
 [6.9 3.1 5.1 2.3]
 [6.4 3.1 4.6 1.8]
 [6.7 3.  5.  1.7]
 [5.6 2.7 4.2 1.3]
 [5.  3.  1.6 0.2]
 [6.  2.2 4.  1. ]
 [5.7 3.8 1.7 0.3]
 [6.7 3.1 4.4 1.4]
 [7.4 2.8 1.5 1.9]
 [6.7 3.3 5.  2.1]
 [6.8 3.2 4.9 2.3]
 [4.6 3.1 1.5 0.2]
 [5.5 2.4 3.8 1.1]
 [6.  2.2 4.  1. ]
 [4.9 3.6 1.4 0.1]
 [5.4 3.7 1.5 0.2]
 [4.9 3.  1.4 0.2]
 [6.  2.7 5.1 1.6]
 [5.8 2.7 3.9 1.2]
 [6.4 2.7 1.4 1.9]
 [7.7 2.6 1.4 2.3]
 [5.  3.  1.6 0.2]
 [7

present: [9]
conditions_eq {}
conditions_leq {}
conditions_geq {3: 0}
raw_data [[4.6 3.1 1.5 1.3]
 [5.9 3.  4.2 1.5]
 [5.4 3.4 1.7 2.5]
 [4.8 3.4 1.6 1.8]
 [5.  3.6 1.4 1.8]
 [4.9 3.6 1.4 1.1]
 [6.3 2.5 5.  1.9]
 [6.9 3.1 5.4 2.1]
 [7.1 3.  5.9 2.1]
 [5.1 2.5 3.  1.1]
 [5.4 3.4 1.5 0.4]
 [6.7 3.  5.2 2.3]
 [5.8 4.  1.2 2.1]
 [6.2 2.9 4.3 1.3]
 [5.  3.4 1.5 1. ]
 [6.1 2.8 4.  1.3]
 [5.8 2.7 5.1 1.9]
 [5.6 2.5 3.9 1.1]
 [5.4 3.4 1.5 0.4]
 [6.1 3.  4.6 1.4]
 [5.7 3.  4.2 1.2]
 [5.8 2.7 4.1 1. ]
 [6.  2.9 4.5 1.5]
 [5.8 4.  1.2 1.8]
 [5.  3.5 1.6 0.6]
 [5.7 3.8 1.7 0.4]
 [5.4 3.9 1.3 0.4]
 [5.8 2.7 5.1 1.9]
 [4.8 3.  1.4 2.1]
 [6.7 2.5 5.8 1.8]
 [6.3 2.5 5.  1.9]
 [4.9 3.1 1.5 1.8]
 [4.9 2.5 4.5 1.7]
 [6.2 2.8 4.8 1.8]
 [5.7 4.4 1.5 0.4]
 [5.2 2.7 3.9 1.4]
 [6.2 3.4 5.4 2.3]
 [6.4 3.2 4.5 1.5]
 [4.9 3.  1.4 2.3]
 [6.4 3.1 5.5 1.8]
 [4.4 3.2 1.3 0.4]
 [7.7 3.8 6.7 2.2]
 [5.5 2.6 4.4 1.2]
 [5.6 2.5 3.9 1.1]
 [5.7 3.  4.2 1.2]
 [6.8 2.8 4.8 1.4]
 [5.9 3.  4.2 1.5]
 [6.3 2.9 5.6 1.8]
 [5.4 3.9

 [6.4 3.2 4.5 1.5]]
d_raw_data [[3. 3. 3. 3.]
 [0. 0. 1. 1.]
 [1. 3. 3. 0.]
 [0. 0. 3. 0.]
 [1. 3. 3. 1.]
 [0. 1. 2. 0.]
 [2. 3. 3. 3.]
 [0. 1. 1. 0.]
 [3. 1. 3. 2.]
 [2. 1. 3. 2.]
 [0. 0. 1. 1.]
 [2. 2. 2. 2.]
 [3. 1. 3. 2.]
 [2. 1. 2. 2.]
 [3. 1. 3. 3.]
 [2. 2. 2. 2.]
 [3. 1. 3. 2.]
 [0. 1. 1. 0.]
 [0. 3. 2. 0.]
 [2. 1. 2. 2.]
 [2. 0. 2. 2.]
 [0. 2. 3. 0.]
 [3. 2. 2. 2.]
 [0. 3. 1. 0.]
 [3. 2. 2. 2.]
 [3. 2. 1. 2.]
 [3. 2. 3. 3.]
 [3. 1. 3. 3.]
 [0. 0. 1. 1.]
 [0. 3. 3. 0.]
 [3. 1. 3. 3.]
 [0. 2. 2. 0.]
 [0. 3. 3. 0.]
 [0. 3. 3. 0.]
 [0. 1. 1. 0.]
 [0. 1. 3. 0.]
 [0. 0. 2. 0.]
 [2. 1. 1. 1.]
 [0. 2. 1. 0.]
 [2. 0. 2. 2.]
 [1. 0. 2. 3.]
 [1. 1. 2. 2.]
 [2. 1. 2. 2.]
 [3. 2. 3. 3.]
 [2. 2. 3. 3.]
 [2. 1. 2. 2.]
 [2. 3. 2. 2.]
 [0. 3. 1. 0.]
 [2. 0. 2. 2.]
 [0. 3. 2. 1.]
 [0. 3. 2. 0.]
 [1. 3. 1. 1.]
 [3. 1. 3. 3.]
 [0. 2. 1. 0.]
 [2. 2. 2. 2.]
 [3. 2. 2. 2.]
 [0. 3. 1. 0.]
 [2. 0. 2. 2.]
 [1. 3. 3. 0.]
 [1. 3. 1. 0.]
 [0. 3. 2. 1.]
 [1. 3. 1. 0.]
 [0. 3. 2. 1.]
 [2. 0. 2. 2.]
 [1. 3. 3

In [306]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())

print('Coverage: %.2f' % exp.coverage())

Anchor: petal length (cm) > 5.10 AND petal width (cm) > 1.80
Precision: 1.00
Coverage: 0.17


In [297]:
exp.features()

[2]

In [240]:


# Get test examples where the anchora pplies
fit_anchor = np.where(np.all(test[:, exp.features()] == test[idx][exp.features()], axis=1))[0]
print('Anchor test precision: %.2f' % (np.mean(blackbox_model.predict(test[fit_anchor]) == blackbox_model.predict(test[idx].reshape(1, -1)))))
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(test.shape[0])))

Anchor test precision: 1.00
Anchor test coverage: 0.03


In [241]:

print('Partial anchor: %s' % (' AND '.join(exp.names(1))))
print('Partial precision: %.2f' % exp.precision(1))
print('Partial coverage: %.2f' % exp.coverage(1))

Partial anchor: petal length (cm) > 5.10 AND sepal length (cm) > 6.40
Partial precision: 1.00
Partial coverage: 0.15


In [231]:
fit_partial = np.where(np.all(test[:, exp.features(1)] == test[idx][exp.features(1)], axis=1))[0]
print('Partial anchor test precision: %.2f' % (np.mean(blackbox_model.predict(test[fit_partial]) == blackbox_model.predict(test[idx].reshape(1, -1)))))
print('Partial anchor test coverage: %.2f' % (fit_partial.shape[0] / float(test.shape[0])))

Partial anchor test precision: 1.00
Partial anchor test coverage: 0.07
